In [2]:
import pandas as pd
import keras as kr
import pandas as pd
from sklearn.model_selection import train_test_split
from matplotlib import pyplot as plt
from keras.utils.np_utils import to_categorical
from keras.models import Sequential
from keras.layers import Dense, Conv2D, Flatten
import csv

Using TensorFlow backend.


In [3]:
train=pd.read_csv("train.csv")
test=pd.read_csv("test.csv")
y=train['label']
x=train.drop(['label'],axis=1)

In [4]:
train

,label,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
41995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41996,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41997,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
41998,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
datagen=kr.preprocessing.image.ImageDataGenerator(rotation_range=10,zoom_range=0.15,width_shift_range=0.1,height_shift_range=0.1)

In [6]:
x=x.values.reshape(-1,28,28,1)
test=test.values.reshape(-1,28,28,1)

In [7]:
x.shape

(42000, 28, 28, 1)

In [8]:
train['label'].unique()

array([1, 0, 4, 7, 3, 5, 8, 9, 2, 6])

In [9]:
y=to_categorical(y,num_classes=10)

In [10]:
x_train,x_val,y_train,y_val=train_test_split(x,y,test_size=0.1,shuffle=True)

In [11]:
datagen.fit(x_train)

In [12]:
cnn=Sequential()
cnn.add(Conv2D(40, kernel_size=3,activation='relu',input_shape=(28,28,1)))
cnn.add(Conv2D(20,kernel_size=3,activation='relu'))
cnn.add(Flatten())
cnn.add(Dense(10,activation='softmax'))

In [13]:
cnn.compile(optimizer='adam',loss='categorical_crossentropy',metrics=['accuracy'])

In [14]:
cnn.fit(x_train, y_train, validation_data=(x_val, y_val), epochs=3)


Train on 37800 samples, validate on 4200 samples
Epoch 1/3
37800/37800 [==============================] - 41s 1ms/step - loss: 0.3666 - accuracy: 0.9383 - val_loss: 0.1148 - val_accuracy: 0.9662
Epoch 2/3
37800/37800 [==============================] - 41s 1ms/step - loss: 0.0851 - accuracy: 0.9745 - val_loss: 0.1120 - val_accuracy: 0.9690
Epoch 3/3
37800/37800 [==============================] - 41s 1ms/step - loss: 0.0585 - accuracy: 0.9823 - val_loss: 0.1303 - val_accuracy: 0.9717


In [15]:
p=cnn.predict(test)

In [16]:
p.shape

(28000, 10)

In [17]:
ans=[]
for i in range(0,28000):
    m=p[i].max()
    for j in range(0,10):
        if p[i][j]==m:
            ans.append(j)       

In [18]:
g=pd.DataFrame(ans)

In [20]:
g['Label']=g[0]

In [21]:
g=g.drop([0],axis=1)

In [28]:
u=[]
for h in range(1,28001):
    u.append(h)

In [29]:
g['ImageId']=u

In [30]:
g

,Label,ImageId
0,2,1
1,0,2
2,9,3
3,0,4
4,3,5
...,...,...
27995,9,27996
27996,7,27997
27997,3,27998
27998,9,27999


In [31]:
h=pd.DataFrame()
h['ImageId']=g['ImageId']
h['Label']=g['Label']

In [33]:
h.to_csv('sample_submission.csv',header=True,index=False,index_label='ImageId')